# GAN (Generative Adversarial Network)
- Generative
  - 생성모델 => 가짜를 만들어내는 모델

    <img src="https://files.slack.com/files-pri/T25783BPY-F9RKVJ9TP/_______________1.png?pub_secret=4674d81e77" width="450px" height="300px">

- Advesarial  
  - 두 개의 모델을 적대적(Adversarial)으로 경쟁시키며 발전시킨다는 것

<img src="https://files.slack.com/files-pri/T25783BPY-F9SHTP6F9/picture2.png?pub_secret=6821873e68" width="450px" height="300px">

GAN은 위조지폐범에 해당하는 생성자(Generator)와 경찰에 해당하는 구분자(Discriminator)를 경쟁적으로 학습시킨다.  
생성자의 목적은 그럴듯한 가짜 데이터를 만들어서 구분자를 속이는 것이며,   
구분자의 목적은 생성자가 만든 가짜 데이터와 진짜 데이터를 구분하는 것이다.  
시간이 흐르면 위조지폐범의 위폐 제조 기술은 완벽에 가깝게 발전할 것이다.


## 실행 코드

### 0. import library

In [0]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pickle

#### GPU check

In [0]:
import os
import imageio

if torch.cuda.is_available():
    use_gpu = True
leave_log = True
if leave_log:
    result_dir = '/content/drive/My Drive/Colab deeplearning/GAN_generated_images'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

### 1. 데이터 로드 & 전처리 방식 지정

In [0]:
#데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
  transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로바꾼다.
  transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

In [0]:
#MNIST 데이터셋을 불러온다. 지정한 폴더에 없을 경우 자동으로 다운로드한다.
dataPath = "/content/drive/My Drive/Colab deeplearning/MNIST"
mnist =datasets.MNIST(root='/content/drive/My Drive/Colab deeplearning/mnist', download=True, transform=transform)

In [0]:
#데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
dataloader =DataLoader(mnist, batch_size=60, shuffle=True)

In [6]:
type(mnist)

torchvision.datasets.mnist.MNIST

In [7]:
mnist[0][0].shape

torch.Size([1, 28, 28])

In [8]:
mnist[0][0][0]

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000

### 2. 모델 구축(생성자 & 구분자)

#### 생성자(Generator) 구축
- 랜덤 벡터 ‘z’를 입력으로 받아 가짜 이미지를 출력하는 함수
  - z : 단순하게 균등 분포(Uniform Distribution)나 정규 분포(Normal Distribution)에서 무작위로 추출된 값
  - z 벡터가 존재하는 공간을 잠재 공간이라 부른다.

  <img src="https://files.slack.com/files-pri/T25783BPY-F9RFJ3VDJ/picture4.png?pub_secret=da0323f283" width="450px" height="300px">

- 선형 레이어
  - 모든 뉴런이 이전 레이어의 모든 뉴런과 연결되는 가장 단순한 구조

#### 사용하는 모델
- 4개의 선형 레이어
  - 100차원 랜덤벡터 => 레이어(256개 뉴런)  
    => 레이어(512) => 레이어(1024) => 마지막 레이어(28x28)
- 활성 함수로는 LeakyReLU ( 0보다 낮은 값은 0.2(설정값)을 곱 )
  - 마지막 레이어 : Tanh ( 픽셀값이 -1~1 이기 때문에 )

In [0]:
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.
class Generator(nn.Module):

  # 네트워크 구조
    def __init__(self):
      super(Generator, self).__init__()
      self.main = nn.Sequential(
        nn.Linear(in_features=100, out_features=256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(in_features=256, out_features=512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(in_features=512, out_features=1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(in_features=1024, out_features=28*28),
        nn.Tanh())
    
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
      return self.main(inputs).view(-1, 1, 28, 28)

In [0]:
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.
class Generator(nn.Module):

  # 네트워크 구조
    def __init__(self):
      super(Generator, self).__init__()
      self.main = nn.Sequential(
        nn.Linear(in_features=100, out_features=256),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=256, out_features=512),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=512, out_features=1024),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=1024, out_features=28*28),
        nn.Tanh())
    
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
      return self.main(inputs).view(-1, 1, 28, 28)

In [0]:
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.
class Generator(nn.Module):

  # 네트워크 구조
    def __init__(self):
      super(Generator, self).__init__()
      self.main = nn.Sequential(
        nn.Linear(in_features=100, out_features=256),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=256, out_features=512),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=512, out_features=1024),
        nn.LeakyReLU(0.2),
        nn.Linear(in_features=1024, out_features=28*28),
        nn.Tanh())
    
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
      return self.main(inputs).view(-1, 1, 28, 28)

#### 구분자(Discriminator) 구축
  - 이미지를 입력으로 받고 그 이미지가 진짜일 확률을 0과 1 사이의 숫자 하나로 출력하는 함수

#### 사용하는 모델
- 4개의 선형 레이어
  - 이미지(28x28) => 레이어(1024 뉴런)  
    => 레이어(512) => 레이어(256) => 마지막 예측 레이어(1)
- 활성 함수로는 LeakyReLU ( 0보다 낮은 값은 0.2(설정값)을 곱 )
  - 마지막 레이어 : Sigmoid ( 출략값이 0~1 이기 때문에 )
- 드롭아웃 층
  - 과적합(Overfitting, 오버피팅)되는 것을 방지
  - 구분자가 생성자보다 지나치게 빨리 학습되는 것 막기 위함.

In [0]:
# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력한다.
class Discriminator(nn.Module):
    
# 네트워크 구조
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(in_features=28*28, out_features=1024),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(inplace=True),
      nn.Linear(in_features=1024, out_features=512),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(inplace=True),
      nn.Linear(in_features=512, out_features=256),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Dropout(inplace=True),
      nn.Linear(in_features=256, out_features=1),
      nn.Sigmoid())
    
  # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
  # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
  def forward(self, inputs):
    inputs = inputs.view(-1, 28*28)
    return self.main(inputs)

In [0]:
# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력한다.
class Discriminator(nn.Module):
    
# 네트워크 구조
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(in_features=28*28, out_features=1024),
      nn.LeakyReLU(0.2, ),
      nn.Dropout(),
      nn.Linear(in_features=1024, out_features=512),
      nn.LeakyReLU(0.2, ),
      nn.Dropout(),
      nn.Linear(in_features=512, out_features=256),
      nn.LeakyReLU(0.2, ),
      nn.Dropout(),
      nn.Linear(in_features=256, out_features=1),
      nn.Sigmoid())
    
  # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
  # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
  def forward(self, inputs):
    inputs = inputs.view(-1, 28*28)
    return self.main(inputs)

#### 생성자 & 구분자 객체 생성

In [27]:
G = Generator()
D = Discriminator()

# use_gpu=False
if use_gpu:
    print("using_GPU")
    G.cuda()
    D.cuda()

using_GPU


### 손실함수 & 최적화기법 지정

- 손실 함수
  - Binary Cross Entropy Loss function
    - 확률이 정답에 가까워지면 낮아지고, 멀면 높아진다.
    - 낮추는 것이 목표
- 최적화 기법 
  - Adam
    - 매개변수마다 업데이트 속도를 최적으로 조절하는 효육적인 최적화 기법

In [0]:
# Binary Cross Entropy loss
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

### 시각화 함수

In [0]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [30]:
if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = Variable(torch.randn(5 * 5, 100), volatile=True)
if use_gpu:
    z_fixed = z_fixed.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


### 모델 반복학습

- 100 epoch  
  => 데이터셋을 100번 순회
- 60 batch size  
  => epoch마다 60개의 데이터를 가져오겠다.

MNIST 학습 데이터의 개수가 6만개이니 1에폭마다 1000번씩 학습이 이루어지는 셈이다.

In [31]:
from time import time

start = time()
# 데이터셋을 100번 돌며 학습한다.
for epoch in range(100):
    
    if leave_log:
        D_losses = []
        G_losses = []
    
    # 한번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        
        # 데이터를 pytorch의 변수로 변환한다.
        real_data = Variable(real_data)

        ### 구분자 학습시키기

        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
         
        if use_gpu:
            real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()
            
        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
            
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimizer.step()
        
        if leave_log:
            # D_losses.append(D_loss.data[0])
            D_losses.append(D_loss.item())

        # train generator G

        ### 생성자 학습시키기
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)
        
        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()
        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()
        
        if leave_log:
            # G_losses.append(G_loss.data[0])
            G_losses.append(G_loss.item())
    if leave_log:
        # true_positive_rate = (D_result_from_real > 0.5).float().mean().data[0]
        # true_negative_rate = (D_result_from_fake < 0.5).float().mean().data[0]
        true_positive_rate = (D_result_from_real > 0.5).float().mean().item()
        true_negative_rate = (D_result_from_fake < 0.5).float().mean().item()
        base_message = ("Epoch: {epoch:<3d} D Loss: {d_loss:<8.6} G Loss: {g_loss:<8.6} "
                        "True Positive Rate: {tpr:<5.1%} True Negative Rate: {tnr:<5.1%}"
                       )
        message = base_message.format(
                    epoch=epoch,
                    d_loss=sum(D_losses)/len(D_losses),
                    g_loss=sum(G_losses)/len(G_losses),
                    tpr=true_positive_rate,
                    tnr=true_negative_rate
        )
        print(message)
    
    if leave_log:
        fake_data_fixed = G(z_fixed)
        image_path = result_dir + '/epoch{}.png'.format(epoch)
        square_plot(fake_data_fixed.view(25, 28, 28).cpu().data.numpy(), path=image_path)
        generated_images.append(image_path)
    
    if leave_log:
        train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
        train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

torch.save(G.state_dict(), "gan_generator.pkl")
torch.save(D.state_dict(), "gan_discriminator.pkl")
with open('gan_train_history.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/GAN_generation.gif', generated_image_array, fps=5)

print("end_time : ", str(start-time()))

Epoch: 0   D Loss: 0.834779 G Loss: 2.08193  True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 1   D Loss: 0.657114 G Loss: 2.56907  True Positive Rate: 81.7% True Negative Rate: 100.0%
Epoch: 2   D Loss: 0.612643 G Loss: 2.54937  True Positive Rate: 86.7% True Negative Rate: 93.3%
Epoch: 3   D Loss: 0.706754 G Loss: 2.13505  True Positive Rate: 86.7% True Negative Rate: 93.3%
Epoch: 4   D Loss: 0.75937  G Loss: 1.92004  True Positive Rate: 83.3% True Negative Rate: 95.0%
Epoch: 5   D Loss: 0.882905 G Loss: 1.61766  True Positive Rate: 81.7% True Negative Rate: 90.0%
Epoch: 6   D Loss: 0.987596 G Loss: 1.37331  True Positive Rate: 90.0% True Negative Rate: 93.3%
Epoch: 7   D Loss: 1.01964  G Loss: 1.29193  True Positive Rate: 63.3% True Negative Rate: 73.3%
Epoch: 8   D Loss: 1.08434  G Loss: 1.1851   True Positive Rate: 63.3% True Negative Rate: 96.7%
Epoch: 9   D Loss: 1.10501  G Loss: 1.14809  True Positive Rate: 31.7% True Negative Rate: 81.7%
Epoch: 10  D Loss: 1.15249  

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


end_time :  -1532.1076147556305


<hr>

# first run
- Loss의 변화가 거의 없다 시피하다..

~~~
Epoch: 0   D Loss: 1.28488  G Loss: 0.859424 True Positive Rate: 55.0% True Negative Rate: 73.3%
Epoch: 1   D Loss: 1.28413  G Loss: 0.858598 True Positive Rate: 61.7% True Negative Rate: 66.7%
Epoch: 2   D Loss: 1.28431  G Loss: 0.858152 True Positive Rate: 66.7% True Negative Rate: 73.3%
Epoch: 3   D Loss: 1.28527  G Loss: 0.858293 True Positive Rate: 61.7% True Negative Rate: 63.3%
Epoch: 4   D Loss: 1.28188  G Loss: 0.861645 True Positive Rate: 68.3% True Negative Rate: 48.3%
Epoch: 5   D Loss: 1.28678  G Loss: 0.856552 True Positive Rate: 66.7% True Negative Rate: 80.0%
Epoch: 6   D Loss: 1.28599  G Loss: 0.856698 True Positive Rate: 55.0% True Negative Rate: 61.7%
Epoch: 7   D Loss: 1.2889   G Loss: 0.852717 True Positive Rate: 58.3% True Negative Rate: 75.0%
Epoch: 8   D Loss: 1.28584  G Loss: 0.85264  True Positive Rate: 53.3% True Negative Rate: 81.7%
Epoch: 9   D Loss: 1.28204  G Loss: 0.868227 True Positive Rate: 60.0% True Negative Rate: 61.7%
Epoch: 10  D Loss: 1.28223  G Loss: 0.862518 True Positive Rate: 45.0% True Negative Rate: 70.0%
Epoch: 11  D Loss: 1.28387  G Loss: 0.858618 True Positive Rate: 60.0% True Negative Rate: 48.3%
Epoch: 12  D Loss: 1.28693  G Loss: 0.854806 True Positive Rate: 65.0% True Negative Rate: 63.3%
Epoch: 13  D Loss: 1.28463  G Loss: 0.864841 True Positive Rate: 56.7% True Negative Rate: 58.3%
Epoch: 14  D Loss: 1.28875  G Loss: 0.854147 True Positive Rate: 50.0% True Negative Rate: 73.3%
Epoch: 15  D Loss: 1.28705  G Loss: 0.852105 True Positive Rate: 65.0% True Negative Rate: 56.7%
Epoch: 16  D Loss: 1.28775  G Loss: 0.853729 True Positive Rate: 53.3% True Negative Rate: 65.0%
Epoch: 17  D Loss: 1.2865   G Loss: 0.855204 True Positive Rate: 50.0% True Negative Rate: 53.3%
Epoch: 18  D Loss: 1.28987  G Loss: 0.848932 True Positive Rate: 55.0% True Negative Rate: 66.7%
Epoch: 19  D Loss: 1.2877   G Loss: 0.856839 True Positive Rate: 61.7% True Negative Rate: 78.3%
Epoch: 20  D Loss: 1.28806  G Loss: 0.853432 True Positive Rate: 51.7% True Negative Rate: 66.7%
Epoch: 21  D Loss: 1.28827  G Loss: 0.852255 True Positive Rate: 65.0% True Negative Rate: 73.3%
Epoch: 22  D Loss: 1.29219  G Loss: 0.851609 True Positive Rate: 65.0% True Negative Rate: 66.7%
Epoch: 23  D Loss: 1.29008  G Loss: 0.853978 True Positive Rate: 61.7% True Negative Rate: 55.0%
Epoch: 24  D Loss: 1.28422  G Loss: 0.858663 True Positive Rate: 68.3% True Negative Rate: 66.7%
Epoch: 25  D Loss: 1.28867  G Loss: 0.856496 True Positive Rate: 38.3% True Negative Rate: 75.0%
Epoch: 26  D Loss: 1.2885   G Loss: 0.852642 True Positive Rate: 66.7% True Negative Rate: 71.7%
Epoch: 27  D Loss: 1.29227  G Loss: 0.846885 True Positive Rate: 63.3% True Negative Rate: 70.0%
Epoch: 28  D Loss: 1.28761  G Loss: 0.857693 True Positive Rate: 50.0% True Negative Rate: 65.0%
Epoch: 29  D Loss: 1.29003  G Loss: 0.846687 True Positive Rate: 61.7% True Negative Rate: 61.7%
Epoch: 30  D Loss: 1.29096  G Loss: 0.852305 True Positive Rate: 65.0% True Negative Rate: 65.0%
Epoch: 31  D Loss: 1.29236  G Loss: 0.851856 True Positive Rate: 55.0% True Negative Rate: 78.3%
Epoch: 32  D Loss: 1.29017  G Loss: 0.853993 True Positive Rate: 66.7% True Negative Rate: 75.0%
Epoch: 33  D Loss: 1.29198  G Loss: 0.849653 True Positive Rate: 56.7% True Negative Rate: 66.7%
Epoch: 34  D Loss: 1.29369  G Loss: 0.842581 True Positive Rate: 40.0% True Negative Rate: 56.7%
Epoch: 35  D Loss: 1.29274  G Loss: 0.846047 True Positive Rate: 58.3% True Negative Rate: 66.7%
Epoch: 36  D Loss: 1.28996  G Loss: 0.85489  True Positive Rate: 60.0% True Negative Rate: 73.3%
Epoch: 37  D Loss: 1.29057  G Loss: 0.849331 True Positive Rate: 56.7% True Negative Rate: 68.3%
Epoch: 38  D Loss: 1.29054  G Loss: 0.847588 True Positive Rate: 46.7% True Negative Rate: 56.7%
Epoch: 39  D Loss: 1.29042  G Loss: 0.850941 True Positive Rate: 53.3% True Negative Rate: 63.3%
Epoch: 40  D Loss: 1.29425  G Loss: 0.84697  True Positive Rate: 60.0% True Negative Rate: 60.0%
Epoch: 41  D Loss: 1.28904  G Loss: 0.853235 True Positive Rate: 56.7% True Negative Rate: 71.7%
Epoch: 42  D Loss: 1.28986  G Loss: 0.848996 True Positive Rate: 65.0% True Negative Rate: 63.3%
Epoch: 43  D Loss: 1.29054  G Loss: 0.84903  True Positive Rate: 51.7% True Negative Rate: 65.0%
Epoch: 44  D Loss: 1.29249  G Loss: 0.851152 True Positive Rate: 45.0% True Negative Rate: 61.7%
Epoch: 45  D Loss: 1.28995  G Loss: 0.850684 True Positive Rate: 50.0% True Negative Rate: 66.7%
Epoch: 46  D Loss: 1.28776  G Loss: 0.853122 True Positive Rate: 58.3% True Negative Rate: 51.7%
Epoch: 47  D Loss: 1.29712  G Loss: 0.840202 True Positive Rate: 70.0% True Negative Rate: 70.0%
Epoch: 48  D Loss: 1.29242  G Loss: 0.847086 True Positive Rate: 58.3% True Negative Rate: 71.7%
Epoch: 49  D Loss: 1.29554  G Loss: 0.842557 True Positive Rate: 63.3% True Negative Rate: 70.0%
Epoch: 50  D Loss: 1.28994  G Loss: 0.850531 True Positive Rate: 50.0% True Negative Rate: 83.3%
Epoch: 52  D Loss: 1.29003  G Loss: 0.852058 True Positive Rate: 70.0% True Negative Rate: 66.7%
Epoch: 53  D Loss: 1.29533  G Loss: 0.843466 True Positive Rate: 60.0% True Negative Rate: 73.3%
Epoch: 54  D Loss: 1.2958   G Loss: 0.842267 True Positive Rate: 66.7% True Negative Rate: 65.0%
Epoch: 55  D Loss: 1.29475  G Loss: 0.842555 True Positive Rate: 51.7% True Negative Rate: 70.0%
Epoch: 56  D Loss: 1.29129  G Loss: 0.849518 True Positive Rate: 60.0% True Negative Rate: 70.0%
Epoch: 57  D Loss: 1.29485  G Loss: 0.847201 True Positive Rate: 48.3% True Negative Rate: 73.3%
Epoch: 58  D Loss: 1.29413  G Loss: 0.850201 True Positive Rate: 63.3% True Negative Rate: 70.0%
Epoch: 59  D Loss: 1.29145  G Loss: 0.851198 True Positive Rate: 63.3% True Negative Rate: 71.7%
Epoch: 60  D Loss: 1.29564  G Loss: 0.844253 True Positive Rate: 53.3% True Negative Rate: 71.7%
Epoch: 61  D Loss: 1.29594  G Loss: 0.843686 True Positive Rate: 50.0% True Negative Rate: 56.7%
Epoch: 62  D Loss: 1.29773  G Loss: 0.842452 True Positive Rate: 53.3% True Negative Rate: 58.3%
Epoch: 63  D Loss: 1.29497  G Loss: 0.845855 True Positive Rate: 55.0% True Negative Rate: 61.7%
Epoch: 64  D Loss: 1.29622  G Loss: 0.842898 True Positive Rate: 55.0% True Negative Rate: 81.7%
Epoch: 65  D Loss: 1.29531  G Loss: 0.843889 True Positive Rate: 36.7% True Negative Rate: 61.7%
Epoch: 66  D Loss: 1.29516  G Loss: 0.844741 True Positive Rate: 50.0% True Negative Rate: 45.0%
Epoch: 67  D Loss: 1.29463  G Loss: 0.845934 True Positive Rate: 63.3% True Negative Rate: 66.7%
Epoch: 68  D Loss: 1.29364  G Loss: 0.846999 True Positive Rate: 56.7% True Negative Rate: 58.3%
Epoch: 69  D Loss: 1.29156  G Loss: 0.850331 True Positive Rate: 45.0% True Negative Rate: 63.3%
Epoch: 70  D Loss: 1.29029  G Loss: 0.849603 True Positive Rate: 65.0% True Negative Rate: 65.0%
Epoch: 71  D Loss: 1.29617  G Loss: 0.841776 True Positive Rate: 65.0% True Negative Rate: 65.0%
Epoch: 72  D Loss: 1.29618  G Loss: 0.841495 True Positive Rate: 56.7% True Negative Rate: 51.7%
Epoch: 73  D Loss: 1.29573  G Loss: 0.841722 True Positive Rate: 46.7% True Negative Rate: 71.7%
Epoch: 74  D Loss: 1.29578  G Loss: 0.844592 True Positive Rate: 71.7% True Negative Rate: 65.0%
Epoch: 75  D Loss: 1.29558  G Loss: 0.841717 True Positive Rate: 65.0% True Negative Rate: 63.3%
Epoch: 76  D Loss: 1.2992   G Loss: 0.839945 True Positive Rate: 45.0% True Negative Rate: 81.7%
Epoch: 77  D Loss: 1.29674  G Loss: 0.842683 True Positive Rate: 38.3% True Negative Rate: 63.3%
Epoch: 78  D Loss: 1.29457  G Loss: 0.844406 True Positive Rate: 51.7% True Negative Rate: 71.7%
Epoch: 79  D Loss: 1.29384  G Loss: 0.84723  True Positive Rate: 78.3% True Negative Rate: 60.0%
Epoch: 80  D Loss: 1.29754  G Loss: 0.840516 True Positive Rate: 43.3% True Negative Rate: 66.7%
Epoch: 81  D Loss: 1.29538  G Loss: 0.843333 True Positive Rate: 45.0% True Negative Rate: 58.3%
Epoch: 82  D Loss: 1.29656  G Loss: 0.841038 True Positive Rate: 63.3% True Negative Rate: 66.7%
Epoch: 83  D Loss: 1.29666  G Loss: 0.837022 True Positive Rate: 56.7% True Negative Rate: 80.0%
Epoch: 84  D Loss: 1.29536  G Loss: 0.846857 True Positive Rate: 46.7% True Negative Rate: 61.7%
Epoch: 85  D Loss: 1.29781  G Loss: 0.840344 True Positive Rate: 58.3% True Negative Rate: 56.7%
Epoch: 86  D Loss: 1.2941   G Loss: 0.850732 True Positive Rate: 46.7% True Negative Rate: 75.0%
Epoch: 87  D Loss: 1.29563  G Loss: 0.843402 True Positive Rate: 66.7% True Negative Rate: 68.3%
Epoch: 88  D Loss: 1.29804  G Loss: 0.845363 True Positive Rate: 50.0% True Negative Rate: 63.3%
Epoch: 89  D Loss: 1.29695  G Loss: 0.841782 True Positive Rate: 60.0% True Negative Rate: 71.7%
Epoch: 90  D Loss: 1.29569  G Loss: 0.843132 True Positive Rate: 63.3% True Negative Rate: 71.7%
Epoch: 91  D Loss: 1.29813  G Loss: 0.837302 True Positive Rate: 60.0% True Negative Rate: 68.3%
Epoch: 92  D Loss: 1.29727  G Loss: 0.838173 True Positive Rate: 63.3% True Negative Rate: 60.0%
Epoch: 93  D Loss: 1.29404  G Loss: 0.845464 True Positive Rate: 66.7% True Negative Rate: 80.0%
Epoch: 94  D Loss: 1.29605  G Loss: 0.844502 True Positive Rate: 56.7% True Negative Rate: 71.7%
Epoch: 95  D Loss: 1.29922  G Loss: 0.838849 True Positive Rate: 65.0% True Negative Rate: 73.3%
Epoch: 96  D Loss: 1.29795  G Loss: 0.842022 True Positive Rate: 45.0% True Negative Rate: 61.7%
Epoch: 97  D Loss: 1.29823  G Loss: 0.835398 True Positive Rate: 65.0% True Negative Rate: 61.7%
Epoch: 98  D Loss: 1.29526  G Loss: 0.845145 True Positive Rate: 58.3% True Negative Rate: 53.3%
Epoch: 99  D Loss: 1.29671  G Loss: 0.843753 True Positive Rate: 60.0% True Negative Rate: 75.0%
/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
end_time :  -1499.3681178092957

~~~

<hr>

#  second run
앞선 결과와 비슷하다.  
랜덤하긴 하지만 결국 비슷하다.

~~~
Epoch: 0   D Loss: 1.29263  G Loss: 0.845172 True Positive Rate: 46.7% True Negative Rate: 60.0%
Epoch: 1   D Loss: 1.29633  G Loss: 0.845791 True Positive Rate: 68.3% True Negative Rate: 63.3%
Epoch: 2   D Loss: 1.29359  G Loss: 0.849673 True Positive Rate: 51.7% True Negative Rate: 58.3%
Epoch: 3   D Loss: 1.29561  G Loss: 0.842294 True Positive Rate: 60.0% True Negative Rate: 71.7%
Epoch: 4   D Loss: 1.29549  G Loss: 0.847163 True Positive Rate: 48.3% True Negative Rate: 70.0%
Epoch: 5   D Loss: 1.29396  G Loss: 0.84448  True Positive Rate: 48.3% True Negative Rate: 76.7%
Epoch: 6   D Loss: 1.29583  G Loss: 0.843481 True Positive Rate: 55.0% True Negative Rate: 75.0%
Epoch: 7   D Loss: 1.29817  G Loss: 0.841995 True Positive Rate: 61.7% True Negative Rate: 60.0%
Epoch: 8   D Loss: 1.29501  G Loss: 0.843043 True Positive Rate: 58.3% True Negative Rate: 65.0%
Epoch: 9   D Loss: 1.29403  G Loss: 0.844912 True Positive Rate: 48.3% True Negative Rate: 70.0%
Epoch: 10  D Loss: 1.29193  G Loss: 0.847046 True Positive Rate: 46.7% True Negative Rate: 71.7%
Epoch: 11  D Loss: 1.29358  G Loss: 0.845821 True Positive Rate: 51.7% True Negative Rate: 71.7%
Epoch: 12  D Loss: 1.29401  G Loss: 0.845127 True Positive Rate: 56.7% True Negative Rate: 75.0%
Epoch: 13  D Loss: 1.29719  G Loss: 0.842042 True Positive Rate: 60.0% True Negative Rate: 46.7%
Epoch: 14  D Loss: 1.29604  G Loss: 0.843616 True Positive Rate: 65.0% True Negative Rate: 60.0%
Epoch: 15  D Loss: 1.29643  G Loss: 0.843412 True Positive Rate: 51.7% True Negative Rate: 60.0%
Epoch: 16  D Loss: 1.29209  G Loss: 0.846979 True Positive Rate: 48.3% True Negative Rate: 73.3%
Epoch: 17  D Loss: 1.29206  G Loss: 0.847234 True Positive Rate: 46.7% True Negative Rate: 60.0%
Epoch: 18  D Loss: 1.29061  G Loss: 0.851166 True Positive Rate: 61.7% True Negative Rate: 63.3%
Epoch: 19  D Loss: 1.29597  G Loss: 0.845586 True Positive Rate: 58.3% True Negative Rate: 63.3%
Epoch: 20  D Loss: 1.29613  G Loss: 0.843349 True Positive Rate: 56.7% True Negative Rate: 73.3%
Epoch: 21  D Loss: 1.29454  G Loss: 0.8431   True Positive Rate: 63.3% True Negative Rate: 63.3%
Epoch: 22  D Loss: 1.28772  G Loss: 0.853526 True Positive Rate: 55.0% True Negative Rate: 65.0%
Epoch: 23  D Loss: 1.29323  G Loss: 0.849733 True Positive Rate: 46.7% True Negative Rate: 75.0%
Epoch: 24  D Loss: 1.29306  G Loss: 0.849541 True Positive Rate: 68.3% True Negative Rate: 58.3%
Epoch: 25  D Loss: 1.29256  G Loss: 0.850123 True Positive Rate: 61.7% True Negative Rate: 71.7%
Epoch: 26  D Loss: 1.29409  G Loss: 0.846212 True Positive Rate: 56.7% True Negative Rate: 90.0%
Epoch: 27  D Loss: 1.295    G Loss: 0.843389 True Positive Rate: 73.3% True Negative Rate: 65.0%
Epoch: 28  D Loss: 1.29488  G Loss: 0.8463   True Positive Rate: 61.7% True Negative Rate: 71.7%
Epoch: 29  D Loss: 1.29707  G Loss: 0.844724 True Positive Rate: 70.0% True Negative Rate: 65.0%
Epoch: 30  D Loss: 1.29421  G Loss: 0.85187  True Positive Rate: 40.0% True Negative Rate: 80.0%
Epoch: 31  D Loss: 1.29107  G Loss: 0.848582 True Positive Rate: 51.7% True Negative Rate: 73.3%
Epoch: 32  D Loss: 1.29467  G Loss: 0.847119 True Positive Rate: 68.3% True Negative Rate: 48.3%
Epoch: 33  D Loss: 1.2913   G Loss: 0.854978 True Positive Rate: 55.0% True Negative Rate: 60.0%
Epoch: 34  D Loss: 1.29719  G Loss: 0.845909 True Positive Rate: 55.0% True Negative Rate: 70.0%
Epoch: 35  D Loss: 1.28971  G Loss: 0.852627 True Positive Rate: 68.3% True Negative Rate: 60.0%
Epoch: 36  D Loss: 1.29121  G Loss: 0.849655 True Positive Rate: 40.0% True Negative Rate: 60.0%
Epoch: 37  D Loss: 1.29379  G Loss: 0.846983 True Positive Rate: 58.3% True Negative Rate: 68.3%
Epoch: 38  D Loss: 1.29     G Loss: 0.852708 True Positive Rate: 65.0% True Negative Rate: 73.3%
Epoch: 39  D Loss: 1.28942  G Loss: 0.855159 True Positive Rate: 61.7% True Negative Rate: 60.0%
Epoch: 40  D Loss: 1.28739  G Loss: 0.859351 True Positive Rate: 60.0% True Negative Rate: 55.0%
Epoch: 41  D Loss: 1.28749  G Loss: 0.857036 True Positive Rate: 63.3% True Negative Rate: 58.3%
Epoch: 42  D Loss: 1.28879  G Loss: 0.856052 True Positive Rate: 43.3% True Negative Rate: 68.3%
Epoch: 43  D Loss: 1.28852  G Loss: 0.854119 True Positive Rate: 58.3% True Negative Rate: 65.0%
Epoch: 44  D Loss: 1.28724  G Loss: 0.859693 True Positive Rate: 50.0% True Negative Rate: 73.3%
Epoch: 45  D Loss: 1.28439  G Loss: 0.866572 True Positive Rate: 61.7% True Negative Rate: 63.3%
Epoch: 46  D Loss: 1.28838  G Loss: 0.859093 True Positive Rate: 63.3% True Negative Rate: 51.7%
Epoch: 47  D Loss: 1.28905  G Loss: 0.852952 True Positive Rate: 58.3% True Negative Rate: 68.3%
Epoch: 48  D Loss: 1.29008  G Loss: 0.855745 True Positive Rate: 58.3% True Negative Rate: 70.0%
Epoch: 49  D Loss: 1.28838  G Loss: 0.859423 True Positive Rate: 58.3% True Negative Rate: 70.0%
Epoch: 50  D Loss: 1.28418  G Loss: 0.865382 True Positive Rate: 51.7% True Negative Rate: 73.3%
Epoch: 51  D Loss: 1.28472  G Loss: 0.857992 True Positive Rate: 51.7% True Negative Rate: 65.0%
Epoch: 52  D Loss: 1.28749  G Loss: 0.860667 True Positive Rate: 53.3% True Negative Rate: 61.7%
Epoch: 53  D Loss: 1.28773  G Loss: 0.860977 True Positive Rate: 65.0% True Negative Rate: 65.0%
Epoch: 54  D Loss: 1.28502  G Loss: 0.858876 True Positive Rate: 66.7% True Negative Rate: 65.0%
Epoch: 55  D Loss: 1.28326  G Loss: 0.861106 True Positive Rate: 60.0% True Negative Rate: 55.0%
Epoch: 56  D Loss: 1.2841   G Loss: 0.867487 True Positive Rate: 60.0% True Negative Rate: 73.3%
Epoch: 57  D Loss: 1.28269  G Loss: 0.867354 True Positive Rate: 68.3% True Negative Rate: 68.3%
Epoch: 58  D Loss: 1.2865   G Loss: 0.858417 True Positive Rate: 56.7% True Negative Rate: 70.0%
Epoch: 59  D Loss: 1.28536  G Loss: 0.863669 True Positive Rate: 65.0% True Negative Rate: 61.7%
Epoch: 60  D Loss: 1.28457  G Loss: 0.866035 True Positive Rate: 56.7% True Negative Rate: 70.0%
Epoch: 61  D Loss: 1.28189  G Loss: 0.868037 True Positive Rate: 73.3% True Negative Rate: 68.3%
Epoch: 62  D Loss: 1.28225  G Loss: 0.869222 True Positive Rate: 51.7% True Negative Rate: 60.0%
Epoch: 63  D Loss: 1.28071  G Loss: 0.866888 True Positive Rate: 63.3% True Negative Rate: 80.0%
Epoch: 64  D Loss: 1.28061  G Loss: 0.86873  True Positive Rate: 55.0% True Negative Rate: 58.3%
Epoch: 65  D Loss: 1.27676  G Loss: 0.876193 True Positive Rate: 53.3% True Negative Rate: 60.0%
Epoch: 66  D Loss: 1.28203  G Loss: 0.867817 True Positive Rate: 48.3% True Negative Rate: 58.3%
Epoch: 67  D Loss: 1.28362  G Loss: 0.869537 True Positive Rate: 60.0% True Negative Rate: 60.0%
Epoch: 68  D Loss: 1.2832   G Loss: 0.868702 True Positive Rate: 66.7% True Negative Rate: 68.3%
Epoch: 69  D Loss: 1.27986  G Loss: 0.870339 True Positive Rate: 61.7% True Negative Rate: 63.3%
Epoch: 70  D Loss: 1.28437  G Loss: 0.867909 True Positive Rate: 53.3% True Negative Rate: 75.0%
Epoch: 71  D Loss: 1.28     G Loss: 0.872519 True Positive Rate: 56.7% True Negative Rate: 51.7%
Epoch: 72  D Loss: 1.27907  G Loss: 0.87575  True Positive Rate: 70.0% True Negative Rate: 81.7%
Epoch: 73  D Loss: 1.28042  G Loss: 0.873787 True Positive Rate: 58.3% True Negative Rate: 61.7%
Epoch: 74  D Loss: 1.27471  G Loss: 0.879901 True Positive Rate: 61.7% True Negative Rate: 68.3%
Epoch: 75  D Loss: 1.27541  G Loss: 0.878734 True Positive Rate: 53.3% True Negative Rate: 75.0%
Epoch: 76  D Loss: 1.27957  G Loss: 0.875591 True Positive Rate: 56.7% True Negative Rate: 51.7%
Epoch: 77  D Loss: 1.27647  G Loss: 0.881347 True Positive Rate: 51.7% True Negative Rate: 65.0%
Epoch: 78  D Loss: 1.27972  G Loss: 0.871875 True Positive Rate: 65.0% True Negative Rate: 65.0%
Epoch: 79  D Loss: 1.27505  G Loss: 0.878769 True Positive Rate: 55.0% True Negative Rate: 75.0%
Epoch: 80  D Loss: 1.27561  G Loss: 0.879759 True Positive Rate: 48.3% True Negative Rate: 65.0%
Epoch: 81  D Loss: 1.27757  G Loss: 0.876944 True Positive Rate: 66.7% True Negative Rate: 81.7%
Epoch: 82  D Loss: 1.2727   G Loss: 0.880679 True Positive Rate: 45.0% True Negative Rate: 66.7%
Epoch: 83  D Loss: 1.27377  G Loss: 0.884236 True Positive Rate: 48.3% True Negative Rate: 70.0%
Epoch: 84  D Loss: 1.27543  G Loss: 0.879843 True Positive Rate: 73.3% True Negative Rate: 70.0%
Epoch: 85  D Loss: 1.27262  G Loss: 0.883837 True Positive Rate: 53.3% True Negative Rate: 61.7%
Epoch: 86  D Loss: 1.27557  G Loss: 0.881245 True Positive Rate: 46.7% True Negative Rate: 71.7%
Epoch: 87  D Loss: 1.27354  G Loss: 0.888755 True Positive Rate: 68.3% True Negative Rate: 71.7%
Epoch: 88  D Loss: 1.27591  G Loss: 0.882658 True Positive Rate: 51.7% True Negative Rate: 53.3%
Epoch: 89  D Loss: 1.27543  G Loss: 0.880732 True Positive Rate: 80.0% True Negative Rate: 65.0%
Epoch: 90  D Loss: 1.27581  G Loss: 0.878049 True Positive Rate: 50.0% True Negative Rate: 78.3%
Epoch: 91  D Loss: 1.27229  G Loss: 0.882451 True Positive Rate: 55.0% True Negative Rate: 70.0%
Epoch: 92  D Loss: 1.27369  G Loss: 0.884049 True Positive Rate: 56.7% True Negative Rate: 78.3%
Epoch: 93  D Loss: 1.27024  G Loss: 0.886449 True Positive Rate: 56.7% True Negative Rate: 75.0%
Epoch: 94  D Loss: 1.27359  G Loss: 0.885341 True Positive Rate: 63.3% True Negative Rate: 61.7%
Epoch: 95  D Loss: 1.27411  G Loss: 0.879282 True Positive Rate: 51.7% True Negative Rate: 80.0%
Epoch: 96  D Loss: 1.27526  G Loss: 0.879899 True Positive Rate: 55.0% True Negative Rate: 55.0%
Epoch: 97  D Loss: 1.27033  G Loss: 0.893547 True Positive Rate: 60.0% True Negative Rate: 80.0%
Epoch: 98  D Loss: 1.27808  G Loss: 0.879218 True Positive Rate: 43.3% True Negative Rate: 71.7%
Epoch: 99  D Loss: 1.27266  G Loss: 0.882205 True Positive Rate: 70.0% True Negative Rate: 70.0%
/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
end_time :  1481.5514345169067
~~~